In [1]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import accelerate
from transformers import pipeline

/home/ltnga/LawVN-Instructction-Gen/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )

Loading checkpoint shards: 100%|██████████| 8/8 [07:05<00:00, 53.16s/it]


In [22]:
file_path = "/home/ltnga/LawVN-Instructction-Gen/src/data/data_gen.json"
with open(file_path, 'r', encoding='utf-8') as f:
        data =json.load(f)

In [23]:
data['content'][0]

'Điều 34. Xử phạt, trừ điểm giấy phép lái xe của người điều khiển quá khổ giới hạn, xe quá tải trọng, xe bánh xích lưu hành đường bộ (kể cả xe ô tô chở hành khách)\n1. Phạt tiền từ 3.000.000 đồng đến 5.000.000 đồng đối với hành vi không thực hiện đúng quy định trong giấy phép lưu hành, trừ các hành vi vi phạm quy định tại điểm a, điểm b khoản 3; điểm b, điểm c khoản 4 Điều này.\n2. Phạt tiền từ 4.000.000 đồng đến 6.000.000 đồng đối với hành vi điều khiển xe mà tổng trọng lượng (khối lượng toàn bộ) của xe vượt quá tải trọng cho phép của đường bộ trên 10% đến 20%, trừ trường hợp có giấy phép lưu hành còn giá trị sử dụng.\n3. Phạt tiền từ 8.000.000 đồng đến 10.000.000 đồng đối với một trong các hành vi vi phạm sau đây:\na) Chở hàng vượt khổ giới hạn của xe hoặc của đường bộ ghi trong giấy phép lưu hành;\nb) Điều khiển xe bánh xích tham gia giao thông không có giấy phép lưu hành hoặc có giấy phép lưu hành nhưng không còn giá trị sử dụng theo quy định hoặc lưu thông trực tiếp trên đường mà 

In [38]:
import re
from rapidfuzz import fuzz
def find_relevant_context(answer: str, full_text: str) -> str:
    """
    Tìm đoạn văn bản context chứa thông tin liên quan đến câu trả lời một cách chính xác hơn
    """
    # Tìm các con số trong câu trả lời (ví dụ: mức phạt)
    numbers = re.findall(r'\d+(?:\.\d+)?', answer)
    
    # Tìm các đoạn văn bản có chứa các con số này
    potential_contexts = []
    for number in numbers:
        # Tìm điều khoản chứa số tiền phạt
        pattern = f"(Điều \d+[\s\S]*?{number}[\s\S]*?đồng.*?\.)"
        matches = re.findall(pattern, full_text)
        potential_contexts.extend(matches)

    if potential_contexts:
        # Lấy context dài nhất (thường là đầy đủ nhất)
        context = max(potential_contexts, key=len)
        # Làm sạch context
        context = re.sub(r'\s+', ' ', context).strip()
        return context

    # Nếu không tìm thấy theo số tiền, tìm theo từ khóa
    keywords = re.findall(r'\b\w+\b', answer.lower())
    best_context = ""
    max_matches = 0
    
    # Tìm điều khoản theo từ khóa
    for match in re.finditer(r'(Điều \d+[\s\S]*?)(?=Điều \d+|$)', full_text):
        section = match.group(1)
        matches = sum(1 for keyword in keywords if keyword in section.lower())
        if matches > max_matches:
            max_matches = matches
            best_context = section

    if best_context:
        # Làm sạch context
        best_context = re.sub(r'\s+', ' ', best_context).strip()
        return best_context
        
    return ""

def extract_qa_pairs(generated_text: str, source_text: str):
    """
    Tách các cặp hỏi-đáp với format chuẩn hơn
    """
    pattern = r"Câu hỏi:\s*(.*?)\s*Trả lời:\s*(.*?)(?=\s*Câu hỏi:|$)"
    matches = re.findall(pattern, generated_text, flags=re.DOTALL)

    results = []
    for q_content, a_content in matches:
        # Làm sạch câu hỏi và câu trả lời
        q_clean = clean_text(q_content)
        a_clean = clean_text(a_content)
        
        # Đảm bảo câu trả lời bắt đầu bằng "Theo Nghị định 168"
        if not a_clean.startswith("Theo Nghị định 168"):
            a_clean = "Theo Nghị định 168, " + a_clean
            
        # Tìm context phù hợp
        context = find_relevant_context(a_clean, source_text)
        if not context:
            continue
            
        results.append({
            "question": q_clean,
            "context": context,
            "answer": a_clean
        })
    return results

def clean_text(text: str) -> str:
    """
    Làm sạch văn bản kỹ hơn
    """
    # Xóa các ký tự đặc biệt
    text = re.sub(r"---+", "", text)
    text = re.sub(r"#+", "", text)
    
    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text)
    
    # Xóa dấu chấm cuối nếu có
    text = text.strip().rstrip('.')
    
    # Thêm dấu chấm cuối nếu chưa có
    if not text.endswith('.'):
        text += '.'
        
    return text

def is_similar(q1: str, q2: str, threshold=70) -> bool:
    ratio = fuzz.ratio(q1.lower(), q2.lower())
    return ratio >= threshold

In [33]:
prompt_template = """
Bạn là một chuyên gia phân tích văn bản pháp luật Việt Nam. Nhiệm vụ của bạn là tạo các cặp câu hỏi và trả lời chuyên sâu về Nghị định 168 về xử phạt giao thông. 

### Văn bản cung cấp:
{context}

### Yêu cầu về output:
- Output phải chứa 3 phần: question (câu hỏi), context (trích dẫn nghị định), answer (câu trả lời)
- Context phải là trích dẫn chính xác từ nghị định, bao gồm số điều khoản
- Answer phải bắt đầu bằng "Theo Nghị định 168,"

### Quy tắc tạo Question:
1. Tập trung vào các nội dung:
   - Mức phạt tiền cho các vi phạm cụ thể
   - Các hành vi bị nghiêm cấm
   - Biện pháp xử phạt bổ sung
   - Thẩm quyền xử phạt
   - Thủ tục xử phạt

2. Format câu hỏi:
   - Ngắn gọn, không quá 20 từ
   - Phải chứa từ khóa cụ thể về hành vi/vi phạm
   - Nên bắt đầu bằng "Mức phạt..." nếu hỏi về tiền phạt

### Quy tắc tạo Context:
- Trích dẫn chính xác điều khoản liên quan
- Format: "Điều X khoản Y: [nội dung]"
- Chỉ lấy phần văn bản liên quan trực tiếp

### Quy tắc tạo Answer:
- Bắt đầu bằng "Theo Nghị định 168,"
- Nêu rõ mức phạt/hình thức xử phạt
- Ngắn gọn, không quá 50 từ
- Kết hợp thông tin từ các điều khoản liên quan

### Ví dụ mẫu:
Câu hỏi: Mức phạt cho hành vi không chấp hành biển báo hiệu giao thông?

Context: "Phạt tiền từ 400.000 đồng đến 600.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: a) Không chấp hành hiệu lệnh, chỉ dẫn của biển báo hiệu, vạch kẻ đường, trừ các hành vi vi phạm quy định tại điểm a, điểm c, điểm d, điểm đ khoản 2; điểm a, điểm d, điểm đ, điểm e, điểm n, điểm o khoản 3;..."

Trả lời: Theo Nghị định 168, hành vi không chấp hành hiệu lệnh, chỉ dẫn của biển báo hiệu, vạch kẻ đường bị phạt tiền từ 400.000 đồng đến 600.000 đồng.

Hãy tạo một cặp question-context-answer duy nhất theo đúng format trên.
"""

In [39]:
text = data['content'][0]  # Lấy phần tử đầu tiên của list

qa_pairs = []
existing_questions = []
num_qa_pairs = 5

for _ in range(num_qa_pairs):
    # Sử dụng text trực tiếp
    prompt = prompt_template.format(context=text)
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=1000,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_content = response.split(prompt)[-1].strip()
    
    pairs = extract_qa_pairs(generated_content,text)
    
    if pairs:
        for pair in pairs:
            q_current = pair['question']
            is_duplicate = any(is_similar(q_current, q_exist) for q_exist in existing_questions)
            
            if not is_duplicate and pair['context']:
                existing_questions.append(q_current)
                qa_pairs.append({
                    "question": pair['question'],
                    "context": pair['context'],
                    "answer": pair['answer']
                })

# Lưu kết quả
output_file = "training_data.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=4)

print(f"Đã lưu {len(qa_pairs)} cặp Q&A vào file {output_file}")

Đã lưu 2 cặp Q&A vào file training_data.json
